In [1]:
%cd ..
%pip install -e .

/home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
Obtaining file:///home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
  Preparing metadata (setup.py) ... done
  Attempting uninstall: src
    Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0
  Running setup.py develop for src
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import torch
import torch.nn.functional as F

P = torch.tensor([
    [.1,.4,.1,.2],
    [.3,.4,.5,.6],
    [.7,.3,.2,.1]
    ], dtype=torch.float)

Q = torch.tensor([
    [.3,.4,.2,.3],
    [.4,.5,.6,.1],
    [.1,.3,.4,.2]
    ], dtype=torch.float)

Y = torch.tensor([1,3,0])

phi_0 = - (P * torch.log(Q)).sum()
phi_1 = F.cross_entropy(P, Y, reduction='sum')

print(phi_0)
print(phi_1)


tensor(5.3950)
tensor(3.4753)


In [3]:
P = torch.tensor([
    [.1,.4,.1,.2],
    [.3,.4,.5,.6],
    [.7,.3,.2,.1]
    ], dtype=torch.float)

Q = torch.tensor([
    [.3,.4,.2,.3],
    [.4,.5,.6,.1],
    [.1,.3,.4,.2]
    ], dtype=torch.float)

Y = torch.tensor([1,3,0])

custome_crossentropy = lambda P, Q: - (P * torch.log(Q)).sum()

print(torch.nn.CrossEntropyLoss(reduction='sum')(P,Q))
print(torch.nn.functional.cross_entropy(P,Q, reduction='sum'))
print(custome_crossentropy(P,Q))

tensor(5.3896)
tensor(5.3896)
tensor(5.3950)


In [16]:
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from notebooks.FinalAnalysisAbdul.models import MLP, GCN, GAT
from notebooks.FinalAnalysisAbdul.p_reg_loss import A_hat_computations


seed = 0

# set up torch env and dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = Planetoid(root=f'data/Planetoid', name='Cora', transform=NormalizeFeatures())
data = dataset[0].to(device)

criterion = torch.nn.CrossEntropyLoss()
model = GCN(dataset, hidden_channels=16, seed = seed).to(device)

_, Z = model(data)

L_preg_0 = lambda x, y: - (x * torch.log(y)).sum()
L_preg_1 = lambda x, y: F.cross_entropy(x, y, reduction='sum')

A_hat, A_hat_mask, N = A_hat_computations(data)
data.reg_mask = torch.ones_like(data.train_mask, dtype=torch.bool)

P = F.softmax(Z, dim=1)
Q = F.softmax(torch.matmul(A_hat, Z), dim=1)

print(L_preg_0((P/1000)[data.reg_mask], (Q/1000)[data.reg_mask]))
print(L_preg_1((P/1000)[data.reg_mask], (Q/1000)[data.reg_mask]))

tensor(23.9757, grad_fn=<NegBackward0>)
tensor(5.2695, grad_fn=<NegBackward0>)


In [18]:
data.y.shape

torch.Size([2708])